# Goal Switching HW #1: Maximum Likelihood Estimation

The goal of maximum likelihood estimation (MLE) is to take a given type of model, and find the parameters that make that model best fit the data. Here, we will use softmax as a practice example. In softmax, the only parameter is Beta (Inverse Tempurature). Here, we will define a `Softmax` class that is a subclass of the `Policy` class. We will define methods for the `Softmax` class such that we can perform maximum likelihood estimation. 

After this assignment, we will be performing MLE on multiple models to see which one describes the given data set the best. We will also come up with our own models. 

The exercises will (tentatively) follow the following progression, which will add sequentially more novel material.

1. Perform Maximum Likelihood Estimation with a Single Model
2. Model Comparison Across Models
3. Model Comparison with Simulated Data
4. Simulation: Goals without Abandonment
5. Simulation: Multi-Faceted Goals
6. Simulation: Open-Ended Goals


First import the necessary libraries. 

In [6]:
import numpy as np
import pandas as pd
import random
from typing import Sequence, Hashable, TypeVar, Tuple, Generic, Container, Set, List, Optional, Union, Dict
from collections import defaultdict
# from scipy.special import logsumexp
# import math
from scipy.optimize import minimize
# from tqdm import tqdm, trange
from dataclasses import dataclass


### Inspect the data
The first thing to note is that the data is in tidy format, with one row per trial. This makes it easy to work with the data. This is standard in science these days. For more about tidy data, see [this site](https://www.jeannicholashould.com/tidy-data-in-python.html).

In [2]:
# load the data from the csv file
data = pd.read_csv('lab_challenge_full.csv')

# dropping some columns i don't think we'll need
data = data.drop(columns=['in_scanner', 
                          'myopic_values_1', 
                          'myopic_values_2', 
                          'myopic_values_3',
                          'tree_search_item1', 
                          'tree_search_item2', 
                          'tree_search_item3',
                          'best_alt_idx',
                          'best_alt_idx',
                          'worst_alt_idx',
                          'best_alt_offer',
                          'worst_alt_offer'
                          ])

# take a look at the first few rows of the data
data.head()



,Unnamed: 0,participant,block,net_trial,net_size,net_contents,offer_1,offer_2,offer_3,choice,switch_trial,prop_filled,raw_distance,trials_invested,net_item_idx,goal_offer,decisionRT
0,0,0,1,1,40,0.000000,5.295200,5.270198,4.559294,1,0,0.000000,40.000000,0,NaN,NaN,NaN
1,1,0,1,2,40,5.295200,5.479601,5.344058,3.756756,1,0,0.132380,34.704800,1,1.0,5.479601,NaN
2,2,0,1,3,40,10.774801,1.702009,4.132068,4.286096,3,1,0.269370,29.225199,2,1.0,1.702009,NaN
3,3,0,1,4,40,4.286096,2.396069,0.924624,4.923793,3,0,0.107152,35.713904,1,3.0,4.923793,NaN
4,4,0,1,5,40,9.209889,2.854140,-0.937678,4.203424,3,0,0.230247,30.790111,2,3.0,4.203424,NaN


For now, we will only need to focus on the following columns:
- `participant`: the participant number
- `block`: the block number which advances each time the participant fills a net with one type of fish
- `net_trial`: the number of trials within the given block
- `net_size`: the size of the net that the participant tries to fill with **one** type of fish/offer
- `net_contents`: how full the net currently is. 
- `offer_X`: each of the three offers made to the participant corresponding to the three goal options

### Define Supporting Classes
We will use these to define a probability distribution over actions. These classes are provided for you, and do not need filling out.

In [8]:
class Distribution:
    def sample(self, rng : random.Random = random):
        raise NotImplementedError
    def prob(self, x):
        raise NotImplementedError
    
class DiscreteDistribution(Distribution):
    support : List
    def __init__(self, support : List, probs : List):
        assert len(support) == len(probs), "Support and probabilities must be the same length"
        assert abs(sum(probs) - 1) < 1e-6, "Probabilities must sum to 1"
        items = defaultdict(float)
        for x, p in zip(support, probs):
            items[x] += p
        support, probs = zip(*items.items())
        self.support = support
        self.probs = probs
    def sample(self, rng : random.Random = random):
        return rng.choices(self.support, weights=self.probs)[0]
    def prob(self, x):
        return self.probs[self.support.index(x)]
    def items(self):
        return zip(self.support, self.probs)
    def __repr__(self) -> str:
        return f"Discrete(support={self.support}, probs={self.probs})"
    def asdict(self) -> Dict:
        return {e: p for e, p in self.items() if p > 0}

### Define state and action representations
Different policies will have different state representations. For a softmax policy, the only information that the state representation will need is each of the three offers on each trial. You will also want to define the action space ahead of time. 

In [9]:
# For Epsilon Greedy, the relevant state on each trial will just be each offer. 
@dataclass(frozen=True)  # frozen=True makes it immutable
class State:
    """Represents a state with three offer values.
    
    Attributes:
        offer1 (float): First offer value
        offer2 (float): Second offer value
        offer3 (float): Third offer value
    """
    offer1: float
    offer2: float
    offer3: float
    
    def __post_init__(self):
        """Validates the state values after initialization."""
        # Check types
        if not all(isinstance(x, (int, float)) for x in (self.offer1, self.offer2, self.offer3)):
            raise TypeError("All offers must be numbers (int or float)")
    
    @property
    def as_tuple(self) -> Tuple[float, float, float]:
        """Returns offers as a tuple."""
        return (self.offer1, self.offer2, self.offer3)
    
    @property
    def as_array(self) -> np.ndarray:
        """Returns offers as a numpy array."""
        return np.array([self.offer1, self.offer2, self.offer3])

# We will also define the action space. We do so as an immutable class with the dataclass decorator
@dataclass(frozen=True)
class ActionSpace:
    actions: Tuple[int, int, int]
    
    def __post_init__(self):
        if not isinstance(self.actions, tuple):
            raise TypeError("Actions must be a tuple")
        if len(self.actions) != 3:
            raise ValueError("Must have exactly 3 actions")
        if not all(isinstance(a, int) for a in self.actions):
            raise TypeError("All actions must be integers")
        if not all(0 <= a <= 2 for a in self.actions):
            raise ValueError("All actions must be 0, 1, or 2")

action_space = ActionSpace((0,1,2))


### Define a `Policy` class
This will serve as the parent class for our `Softmax` class. Notice that we will be able to use this for most any policy that we might come up with. We will be reusing the parent class to define multiple kinds of policies. We will use this class to define the different models we will fit to the data. No need to implement methods here. 

Notice that many of the methods have already been defined for you, but that some are left undefined. We will fill in the undefined methods in the `Softmax` subclass. The intention here is that as long as the incomplete methods are coded correctly, the rest of the methods in the class will work smoothly for you. 


In [118]:
# Leave this abstract for now. Meaning, don't implement the methods. 
class Policy:
    action_space : Sequence[Hashable]
    
    def __init__(self, name, action_space : ActionSpace):
        '''The constructor for the class objects.'''
        self.name = name
        self.action_space = action_space
    
    def action_dist(self, s : State, params : List) -> Distribution:
        '''
        The policy defines a distribution over actions for a given state.
        
        Args:
            s: The current state.
            params: A list of relevant parameters. (e.g. epsilon in epsilon greedy)

        Returns:
            action_dist: a dictionary of keys that are actions and the values are the probability of taking those actions

        '''
        # Implement this method later for specific policies. 
        raise NotImplementedError
    
    def sample_action(self, s : State, params : List, rng : random.Random = random) -> Tuple[int, float]:
        '''Sample an action according to the action distribution and return that action and the likelihhod.'''
        action_dist = self.action_dist(s, params)
        actions, probs = zip(*action_dist.items()) # unpack the dictionary into keys and values
        action = rng.choices(actions, weights=probs, k=1)[0] # sample one action accordingly
        prob = action_dist[action] # get the probability that action was chosen
        return action, prob

    def calc_neg_log_lik_single(self, s : State, a : int, params : List) -> float:
        '''Takes an action and state and calcs the neg log lik that action happened in that state.'''
        action_dist = self.action_dist(s, params).asdict()
        neg_log_lik = - np.log(action_dist[a])
        return neg_log_lik
    
    def _create_state_arr(self, data : pd.DataFrame) -> Sequence[State]:
        raise NotImplementedError
    
    def _create_action_arr(self, data : pd.DataFrame) -> Sequence[int]:
        '''Create an array of all the action taken.'''
        raise NotImplementedError
    
    def calc_neg_log_lik_arr(self, data : pd.DataFrame, params : List) -> Sequence[float]:
        '''Calculate the negloglik for each row in a dataset. '''
        s_arr = self._create_state_arr(data)
        a_arr = self._create_action_arr(data)
        assert len(s_arr)==len(a_arr), f'Action array and state array must be same length.'
        negloglik_arr = np.zeros(len(s_arr))
        negloglik_arr[:] = [self.calc_neg_log_lik_single(s, a, params) 
                    for s, a in zip(s_arr, a_arr)]
        return negloglik_arr
    
    def calc_neg_log_like_total(self, params : List, data : pd.DataFrame, ):
        '''Return the negloglik as a sum total.'''
        return np.sum(self.calc_neg_log_lik_arr(data, params))
    
    def max_lik_estimation(data) -> Tuple[np.ndarray, float]:
        '''Uses the likelihood function in the class to estimate the best params given some set of data.
        
        Args:
            data: DataFrame containing trials with columns for offers and choices
            
        Returns:
            Tuple containing:
                - np.ndarray: Optimal parameters
                - float: Final negative log likelihood
        
        '''

        raise NotImplementedError




### Problem A: Complete the `Softmax` class
This policy uses softmax to choose options based off of their offers. Its *parent* class is the `Policy` class, which it *inherets* methods from. Notice that if we define a method again in the *child* class, it overwrites the method (function) from the parent's class. This is called method *overriding*. Here, we are simply overriding methods that were not completed in the `Policy` class. If you do not complete them, but try to run them, it will throw an approprate error that the methods were `NotImplemented`.

In [119]:
class Softmax(Policy):

    def __init__(self, action_space: ActionSpace):
        super().__init__('Softmax', action_space)
        self.n_params = 1 # just inverse tempurature. 
        
    
    def action_dist(self, s : State, params : List) -> Distribution:
        '''
        The policy defines a distribution over actions for a given state.
        
        Args:
            s: The current state.
            params: A list of relevant parameters. (e.g. epsilon in epsilon greedy)

        Returns:
            action_dist: a dictionary of keys that are actions and the values are the probability of taking those actions

        '''
        ####################################################
        # Implement this method. 
        ####################################################
        raise NotImplementedError


    def _create_state_arr(self, data : pd.DataFrame) -> Sequence[State]:
        '''Create an array of states needed for `action_dist`. One state per trial.'''
        ####################################################
        # Implement this method. 
        ####################################################
        raise NotImplementedError
    
    def _create_action_arr(self, data : pd.DataFrame) -> Sequence[int]:
        '''Create an array of all the actions taken. One action per trial'''
        ####################################################
        # Implement this method. 
        ####################################################
        raise NotImplementedError
        
    def max_lik_estimation(self, data : pd.DataFrame) -> Tuple[float, float]:
        ''' Perform maximum likelihood estimation.

        Args:
            data: DataFrame containing trials with columns for offers and choices

        Returns:
                Tuple of:
                    - Optimal temperature parameter
                    - Final negative log likelihood
        '''

        # we will minimize 5 times, and will take the best of those 5 starts
        initial_conds = np.random.uniform(low=1, high=10, size=5) # we must feed the minimizer some initial conditions
        bounds = [(0.001, None)] # the bounds for inverse tempurature are (0, inf)
        best_result = None
        best_nll = np.inf

        # minimize for each of the generated intial conditions
        for initial_cond in initial_conds:
            result = minimize(
                self.calc_neg_log_like_total, # find the parameter that minimizes negloglik
                args=(data), # this is always passed as a tuple
                x0 = [initial_cond], # initial conditions
                bounds=bounds, # the space to search with the minimization function
                method='L-BFGS-B' # the minimization method. More options out there is this doesn't work well.
            )

            # take the best minimization so far
            if result.success and result.fun < best_nll:
                best_result = result
                best_nll = result.fun

        if best_result is None:
                raise RuntimeError("Optimization failed from all starting points")
        
        return best_result.x[0], best_result.fun # best inv temp, best nll

    

### Tests
Does the code run without errors? There are some assertions that should pass, and will tell you a generic message about what is wrong if they do not pass. 


In [128]:
# create the policy class
softmax_policy = Softmax(action_space)

# we can test on fake trials
state = State(1,2,3)
params = [5] # inverse temp
action_dist_test = softmax_policy.action_dist(state, params).asdict()
print('Example action distribution:')
print(action_dist_test)
print()

# we can also make sure that we are able to calculate neglogliks
negloglik_test1 = softmax_policy.calc_neg_log_lik_single(state, 0, params)
print('Negloglik Test:')
print(negloglik_test1)

# we can also make sure that we are able to calculate neglogliks
negloglik_test2 = softmax_policy.calc_neg_log_lik_single(state, 1, params)
print('Negloglik Test:')
print(negloglik_test2)

# check that all probabilities for a trial sum to 1
assert np.isclose(np.sum(list(action_dist_test.values())), 1), 'For each trial, the sum of the likelihoods of each choice should sum to 1.'

# test that likelihood works as expected
assert negloglik_test1 > negloglik_test2, 'Actions with lower probability should have higher negloglik'

# simple create state/action arr test
assert len(softmax_policy._create_state_arr(data))==len(data), 'State array has wrong length.'
assert len(softmax_policy._create_action_arr(data))==len(data), 'Action array has wrong length'


Example action distribution:
{0: 0.2693074991777379, 1: 0.3289329222889067, 2: 0.4017595785333554}

Negloglik Test:
1.3119014326242002
Negloglik Test:
1.1119014326242003


### Try out calculating negative log likelihood.
The total negloglik should be positive number about in the thousands for this dataset.

In [121]:
softmax_policy.calc_neg_log_like_total(params, data)

7291.406939107112

### Try running maximum likelihood estimation.
This might take a little longer to run, but shouldn't take more than about 1 minute. 

In [127]:
softmax_policy.max_lik_estimation(data)

(5.065322172870126, 7291.216180944285)

## Problem B: Complete Epsilon `Greedy` class
This time, create class for an epsilon greedy policy. First, complete the same methods as you did for `Softmax`. Can you use the same methods as you did before? A hint here is that both policies take very similar information (state representations), but have different parameters (`params`). 

Note that since the polcies are parameterized differently, we will need to re-write the `max_lik_estimation` function. 

In [146]:
class Greedy(Policy):

    def __init__(self, action_space: ActionSpace):
        super().__init__('Softmax', action_space)
        self.n_params = 1 # just inverse tempurature. 
        
    
    def action_dist(self, s : State, params : List) -> Distribution:
        '''
        The policy defines a distribution over actions for a given state.
        
        Args:
            s: The current state.
            params: A list of relevant parameters. (e.g. epsilon in epsilon greedy)

        Returns:
            action_dist: a dictionary of keys that are actions and the values are the probability of taking those actions

        '''
        ####################################################
        # Implement this method. The array params will just have a single element: epsilon.
        ####################################################

        raise NotImplementedError

    def _create_state_arr(self, data : pd.DataFrame) -> Sequence[State]:
        '''Create an array of states needed for `action_dist`. One state per trial.'''
        ####################################################
        # Implement this method. 
        ####################################################
        raise NotImplementedError
    
    def _create_action_arr(self, data : pd.DataFrame) -> Sequence[int]:
        '''Create an array of all the actions taken. One action per trial'''
        ####################################################
        # Implement this method. 
        ####################################################
        raise NotImplementedError
        
    def max_lik_estimation(self, data : pd.DataFrame) -> Tuple[float, float]:
        ''' Perform maximum likelihood estimation.

        Args:
            data: DataFrame containing trials with columns for offers and choices

        Returns:
                Tuple of:
                    - Optimal temperature parameter
                    - Final negative log likelihood
        '''
        ####################################################
        # Implement this method. 
        # Reference the method in the `Softmax` class to scaffold you.
        ####################################################
        
        raise NotImplementedError

    

In [142]:
# create the policy class
greedy_policy = Greedy(action_space)

# we can test on fake trials
state = State(1,2,3)
params = [0.2] # inverse temp
action_dist_test = greedy_policy.action_dist(state, params).asdict()
print('Example action distribution:')
print(action_dist_test)
print()

# we can also make sure that we are able to calculate neglogliks
negloglik_test1 = greedy_policy.calc_neg_log_lik_single(state, 0, params)
print('Negloglik Test:')
print(negloglik_test1)

# we can also make sure that we are able to calculate neglogliks
negloglik_test2 = greedy_policy.calc_neg_log_lik_single(state, 1, params)
print('Negloglik Test:')
print(negloglik_test2)

# we can also make sure that we are able to calculate neglogliks
negloglik_test3 = greedy_policy.calc_neg_log_lik_single(state, 2, params)
print('Negloglik Test:')
print(negloglik_test3)

# check that all probabilities for a trial sum to 1
assert np.isclose(np.sum(list(action_dist_test.values())), 1), 'For each trial, the sum of the likelihoods of each choice should sum to 1.'

# check that epsilon greedy assigns same nll to non-greedy options
assert negloglik_test1==negloglik_test2, 'Non-greedy options should be chosen with equal likelihood.'

# simple create state/action arr test
assert len(greedy_policy._create_state_arr(data))==len(data), 'State array has wrong length.'
assert len(greedy_policy._create_action_arr(data))==len(data), 'Action array has wrong length'

Example action distribution:
{0: 0.1, 1: 0.1, 2: 0.8}

Negloglik Test:
2.3025850929940455
Negloglik Test:
2.3025850929940455
Negloglik Test:
0.2231435513142097


### Check negative log likelihood (nll) calculation.

In [144]:
# does this run okay?
greedy_policy.calc_neg_log_like_total(params, data)

7762.000227607521

### Attempt MLE

In [145]:
greedy_policy.max_lik_estimation(data)

(0.3564417906661573, 7231.302407928132)

## Problem C, Part 1
In Problem B, we performed maximum likelihood estimation for an epsilon greedy policy. It is worth thinking about what espilon means in this context. Specifically, if people were behaving completely randomly on this task, what value might we expect epsilon to take? (Hint: by traditional notions of epsilong greedy, the answer is not `0.33`). Write your thoughts below. 

### Response
...

## Problem C, Part 2
We do not expect people to be behaving randomly on this task. But what if epsilon were to indicate that people were choosing the highest offer on each trial according to chance? (1) Would this imply that people were behaving randomly? (2) If yes, what are ways we could check to see that were the case? If not, what are alternative explanations that epsilon indicated that people were choosing according to chance, but in fact were not?

### Response
...